# Основная часть

In [101]:
import re
import numpy as np
from typing import DefaultDict
from collections import defaultdict

In [102]:
def get_statictics(path: str) -> DefaultDict[str, int]:
    segments = list()
    output = defaultdict(int)

    with open(path, 'r') as stream_reader:
        for line in stream_reader:
            if line.startswith('>'):
                output['overall_count'] += 1

                contig = int(re.findall(r"len(\d+)_", line)[0])
                segments.append(contig)
                output['overall_length'] += contig
    segments = np.array(segments)
    
    sorted_indexes = np.argsort(segments)[::-1]
    output['max_single_length'] = segments[sorted_indexes][0]
    output['max_single_length_idx'] = sorted_indexes[0]

    overall_score = 0.0
    for contig in segments[sorted_indexes]:
        overall_score += contig
        if overall_score >= 0.5 * output['overall_length']:
            output['N50'] = contig
            return output

### Статистика по контигам

In [103]:
contigs_stats = get_statictics("Poil_contig.fa")
print(f"Общее количество контигов: {contigs_stats['overall_count']}")
print(f"Общая длина контигов: {contigs_stats['overall_length']}")
print(f"Длина наибольшего контига: {contigs_stats['max_single_length']}")
print(f"N50: {contigs_stats['N50']}")

Общее количество контигов: 619
Общая длина контигов: 3926043
Длина наибольшего контига: 129537
N50: 49851


### Статистика по скаффолдам

In [104]:
scaffolds_stats = get_statictics("Poil_scaffold.fa")
print(f"Общее количество скаффолдов: {scaffolds_stats['overall_count']}")
print(f"Общая длина скаффолдов: {scaffolds_stats['overall_length']}")
print(f"Длина наибольшего скаффолда: {scaffolds_stats['max_single_length']}")
print(f"N50: {scaffolds_stats['N50']}")

Общее количество скаффолдов: 72
Общая длина скаффолдов: 3876363
Длина наибольшего скаффолда: 3833080
N50: 3833080


In [105]:
def get_sequence(path, start_index):
    sequence_parts = list()
    with open(path, 'r') as stream_reader:
        lines = stream_reader.readlines()
        for line in lines[(start_index + 1):]:
            if line.startswith('>'):
                break
            else:
                sequence_parts.append(line)
    
    return re.sub(r"[\s\n\r]+", "", "".join(sequence_parts))

In [106]:
longest_scaffold = get_sequence("Poil_scaffold.fa", scaffolds_stats['max_single_length_idx'])

with open('longest.fasta', 'w') as stream_writer:
    stream_writer.write(longest_scaffold)

In [107]:
print(f"Общая длина гэпов: {longest_scaffold.count('N')}")

Общая длина гэпов: 7199


In [108]:
longest_scaffold_gaps_tokenized = re.sub(r"N+", "[GAP]", longest_scaffold)
print(f"Количество гэпов: {longest_scaffold_gaps_tokenized.count('[GAP]')}")

Количество гэпов: 64


### Статистика по уменьшенному количеству гэпов

In [109]:
with open("Poil_gapClosed.fa", 'r') as stream_reader:
    max_length = 0
    indexes = list()
    current_length = 0
    for index, line in enumerate(stream_reader):
        if line.startswith('>'):
            if current_length > max_length:
                max_length = current_length
                longest_scaffold_short_gaps_start = indexes[-1]
            current_length = 0
            indexes.append(index)
        else:
            current_length += len(line)

In [110]:
longest_scaffold_short_gaps = get_sequence("Poil_gapClosed.fa", longest_scaffold_short_gaps_start)

In [111]:
print(f"Общая длина гэпов: {longest_scaffold_short_gaps.count('N')}")

Общая длина гэпов: 2298


In [112]:
longest_scaffold_short_gaps_tokenized = re.sub(r"N+", "[GAP]", longest_scaffold_short_gaps)
print(f"Количество гэпов: {longest_scaffold_short_gaps_tokenized.count('[GAP]')}")

Количество гэпов: 9
